Tensors are higher order extensions of matrices that can encode multi-dimensional data

![tensor_illustration](../img/tensor_cartoon.jpg)

In this tutorial we will show how to manipulate tensors as NDArrays, using [TensorLy](http://tensorly.github.io) with the MXNet backend to perform tensor operations.

In [1]:
import numpy as np
import tensorly as tl

Using mxnet backend.


# 1.Creating a tensor

A tensor can be represented in multiple ways. The simplest is the slice representation through multiple matrices.

Let's take for this example the tensor $\tilde X$ defined by its frontal slices:

$$
   X_1 = 
   \left[
   \begin{matrix}
   0  & 2  & 4  & 6\\
   8  & 10 & 12 & 14\\
   16 & 18 & 20 & 22
   \end{matrix}
   \right]
$$

and 

$$
   X_2 =
   \left[
   \begin{matrix}
   1  & 3  & 5  & 7\\
   9  & 11 & 13 & 15\\
   17 & 19 & 21 & 23
   \end{matrix}
   \right]
$$




In Python, this array can be expressed as a numpy array::

In [2]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))

In [3]:
X


[[[  0.   1.]
  [  2.   3.]
  [  4.   5.]
  [  6.   7.]]

 [[  8.   9.]
  [ 10.  11.]
  [ 12.  13.]
  [ 14.  15.]]

 [[ 16.  17.]
  [ 18.  19.]
  [ 20.  21.]
  [ 22.  23.]]]
<NDArray 3x4x2 @cpu(0)>

You can view the frontal slices by fixing the last axis:

In [4]:
X[:, :, 0]


[[  0.   2.   4.   6.]
 [  8.  10.  12.  14.]
 [ 16.  18.  20.  22.]]
<NDArray 3x4 @cpu(0)>

In [5]:
X[:, :, 1]


[[  1.   3.   5.   7.]
 [  9.  11.  13.  15.]
 [ 17.  19.  21.  23.]]
<NDArray 3x4 @cpu(0)>

# 2.Setting the backend

In TensorLy you can dynamically set the backend to use either NumPy or MXNet to represent tensors and perform the operations:

In [6]:
type(X)

mxnet.ndarray.NDArray

By default, the backend is set to MXNet, here is how to change it:

In [7]:
tl.set_backend('numpy')

Using numpy backend.


In [8]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))
type(X)

numpy.ndarray

As expected tensors are now represented as NumPy arrays. 
Let's change it back to MXNet for the rest of the tutorial.

In [9]:
tl.set_backend('mxnet')

Using mxnet backend.


In [10]:
X = tl.tensor(np.arange(24).reshape((3, 4, 2)))
type(X)

mxnet.ndarray.NDArray

# 3.Basic tensor operations

## 3.1 Unfolding

Also called **matrization**, **unfolding** a tensor is done by reading the element in a given way as to obtain a matrix instead of a tensor.

It is done by stacking the **fibers** of the tensor into a matrix.

![tensor_illustration](../img/tensor_fibers.png)
Illustration: *Nonnegative Matrix and Tensor Factorizations*, Andrzej Cichocki, Rafal Zdunek, Anh Huy Phan, and Shun-ichi Amari, John Wiley & Sons, 2009.



### Definition
For a tensor of size $(I_1, I_2, \cdots, I_n)$, the k-mode unfolding of this tensor will be of size $(I_k, I_1 \times \cdots \times I_{k-1} \times I_{k+1} \cdots \times I_n)$.


   Given a tensor $\tilde X \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$, the
   mode-n unfolding of $\tilde X$ is a matrix $\mathbf{X}_{[n]} \in \mathbb{R}^{I_n, I_M}$,
   with $M = \prod\limits_{\substack{k=1,\\k \neq n}}^N I_k$ and is defined by
   the mapping from element $(i_1, i_2, \cdots, i_N)$ to $(i_n, j)$, with


$$
    j = \sum\limits_{\substack{k=1,\\k \neq n}}^N i_k \times \prod_{m=k+1}^N I_m.
$$

### Convention

   Traditionally, mode-1 unfolding denotes the unfolding along the first dimension.
   However, to be consistent with the Python indexing that always starts at zero,
   in tensorly, unfolding also starts at zero!

   Therefore ``unfold(tensor, 0)`` will unfold said tensor along its first dimension!
   

### Example

For instance, using the $\tilde X$ previously defined:
$$
   X_1 = 
   \left[
   \begin{matrix}
   0  & 2  & 4  & 6\\
   8  & 10 & 12 & 14\\
   16 & 18 & 20 & 22
   \end{matrix}
   \right]
$$

and 

$$
   X_2 =
   \left[
   \begin{matrix}
   1  & 3  & 5  & 7\\
   9  & 11 & 13 & 15\\
   17 & 19 & 21 & 23
   \end{matrix}
   \right]
$$

The 0-mode unfolding of $\tilde X$:

$$
   \tilde X_{[0]} =
   \left[ \begin{matrix}
      0 & 1 & 2 & 3 & 4 & 5 & 6 & 7\\
      8 & 9 & 10 & 11 & 12 & 13 & 14 & 15\\
      16 & 17 & 18 & 19 & 20 & 21 & 22 & 23\\
   \end{matrix} \right]
$$

The 1-mode unfolding is given by:

$$
   \tilde X_{[1]} =
   \left[ \begin{matrix}
      0 & 1 & 8 & 9 & 16 & 17\\
      2 & 3 & 10 & 11 & 18 & 19\\
      4 & 5 & 12 & 13 & 20 & 21\\
      6 & 7 & 14 & 15 & 22 & 23\\
   \end{matrix} \right]
$$

Finally, the 2-mode unfolding is the unfolding along the last axis:

$$
    \tilde X_{[2]} =
   \left[ \begin{matrix}
      0 & 2 & 4 & 6 & 8 & 10 & 12 & 14 & 16 & 18 & 20 & 22\\
      1 & 3 & 5 & 7 & 9 & 11 & 13 & 15 & 17 & 19 & 21 & 23\\
   \end{matrix} \right]
$$

### In TensorLy


In [11]:
tl.unfold(X, mode=0)


[[  0.   1.   2.   3.   4.   5.   6.   7.]
 [  8.   9.  10.  11.  12.  13.  14.  15.]
 [ 16.  17.  18.  19.  20.  21.  22.  23.]]
<NDArray 3x8 @cpu(0)>

In [12]:
tl.unfold(X, mode=1)


[[  0.   1.   8.   9.  16.  17.]
 [  2.   3.  10.  11.  18.  19.]
 [  4.   5.  12.  13.  20.  21.]
 [  6.   7.  14.  15.  22.  23.]]
<NDArray 4x6 @cpu(0)>

In [13]:
tl.unfold(X, mode=2)


[[  0.   2.   4.   6.   8.  10.  12.  14.  16.  18.  20.  22.]
 [  1.   3.   5.   7.   9.  11.  13.  15.  17.  19.  21.  23.]]
<NDArray 2x12 @cpu(0)>

## 3.2 Folding

Folding is the inverse operation: you can **fold** an unfolded tensor back from matrix to full tensor using the ``tensorly.fold`` function.

In [14]:
unfolding = tl.unfold(X, 1)
original_shape = X.shape
tl.fold(unfolding, mode=1, shape=original_shape)


[[[  0.   1.]
  [  2.   3.]
  [  4.   5.]
  [  6.   7.]]

 [[  8.   9.]
  [ 10.  11.]
  [ 12.  13.]
  [ 14.  15.]]

 [[ 16.  17.]
  [ 18.  19.]
  [ 20.  21.]
  [ 22.  23.]]]
<NDArray 3x4x2 @cpu(0)>

## 3.3 n-mode product

Also known as **tensor contraction**. This is a natural generalization of matrix-vector and matrix-matrix product. When multiplying a tensor by a matrix or a vector, we now have to specify the **mode** $n$ along which to take the product.
### Tensor times matrix

In that case we are doing an operation analogous to a matrix multiplication on the $n$-th mode. Given a tensor $\tilde X$ of size $(I_1, I_2, \cdots, I_N)$, and a matrix $M$ of size $(D, I_n)$, the $n$-mode product of $\tilde X$ by $M$ is written $\tilde X \times_n M$ and is of size $(I_k, I_1 \times \cdots \times I_{n-1} \times D \times I_{n+1} \cdots \times I_n)$.

### Tensor times vector

In that case we are contracting over the $n$-th mode by multiplying it with a vector. Given a tensor $\tilde X$ of size $(I_1, I_2, \cdots, I_N)$, and a vector $v$ of size $(I_n)$, the $n$-mode product of $\tilde X$ by $v$ is written $\tilde X \times_n v$ and is of size $(I_k, I_1 \times \cdots \times I_{n-1} \times I_{n+1} \cdots \times I_n)$.

![tensor_illustration](../img/tensor_contraction.png)


### Example

In TensorLy, all the tensor algebra functions are located in the `tensorly.tenalg` module. For the n-mode product, you will need to use the function `mode_dot` that works transparently for multiplying a tensor by a matrix or a vector along a given mode.

#### Tensor times matrix

With the tensor $\tilde X$ of size (3, 4, 2) we defined previously, let's define a matrix M of size (5, 4) to multiply along the second mode:

In [15]:
M = tl.tensor(np.arange(4*5).reshape((5, 4)))
print(M.shape)

(5, 4)


Keep in mind indexing starts at zero, so the second mode is represented by `mode=1`:

In [16]:
res = tl.tenalg.mode_dot(X, M, mode=1)

As expected the result is of shape (3, 5, 2)

In [17]:
res.shape

(3, 5, 2)

#### Tensor times vector

Similarly, for a vector of size 4 and contract along the mode 1.

In [18]:
v = tl.tensor(np.arange(4))
print(v.shape)

(4,)


In [19]:
res = tl.tenalg.mode_dot(X, v, mode=1)

Since we have multiplied by a vector, we have effectively contracted out one mode of the tensor so the result is a matrix:

In [20]:
res.shape

(3, 2)